# Part 2: Introduction to MCMC and emcee

In this notebook we will go over what MCMC means and what exactly is it that is going on when we run an MCMC algorithm such as Metropolist-Hasting or a code like emcee. 

## What is MCMC?

Markov Chain Monte Carlo (MCMC) is a class of algorithms used to sample from a probability distribution. It is particularly useful when dealing with high-dimensional spaces where traditional sampling methods are inefficient. MCMC methods construct a Markov chain that has the desired distribution as its equilibrium distribution. By simulating the Markov chain, we can obtain samples from the target distribution. As astronomers we do a lot of work trying to estimate physical paramters on astrophysical sources such as stars, galaxies and cosmology. Most of the time the target distribution we are after is getting a handle on the parameters for a certain model and the uncertainties on those parameters. MCMC comes in handy to sample the parameter spaces for these complex models and allows us to see how constrained certain parameters are. 

### Key Concepts

- **Markov Chain**: A sequence of random variables where the distribution of each variable depends only on the state of the previous one.
- **Monte Carlo**: Refers to the use of random sampling to estimate numerical results.
- **Equilibrium Distribution**: The distribution to which the Markov chain converges after a large number of steps.

### Common MCMC Algorithms

- **Metropolis-Hastings**: A widely used MCMC algorithm that generates a sequence of samples by proposing moves to new states and accepting or rejecting them based on a certain probability.
- **Gibbs Sampling**: A special case of the Metropolis-Hastings algorithm where each variable is sampled from its conditional distribution given the current values of the other variables.

### Applications

MCMC methods are used in various fields such as:

- Bayesian statistics for posterior distribution sampling
- Computational physics for simulating systems with many degrees of freedom
- Machine learning for training models with complex likelihood functions


# Why do we use MCMC to begin with? 

# Introducing Bayesian Statistics

The reason why we go ahead and use MCMC to begin with is because we are using a Bayesian framework to understand the features we are studying. To start our Bayesian journey we need to go over the main equation that is driving Bayesian Statistics forward, Bayes Theorem:

$P(\theta | x) = \frac{P(x|\theta)P(\theta)}{P(x)}$

Let us unpack the above equation:

$P(\theta|x)$ is the quantity we are trying to find which is the probability of acquiring certain parameters $\theta$ provided the data, $x$. In Bayesian terms this is the *Posterior* Distribution. $P(x|\theta)P(\theta)$ is a combination of the *likelihood* function $P(x|\theta)$, which is the probability of acquiring the data for a set of input quantities $\theta$ and $P(\theta)$ is the *prior* which is the probability of getting those parameters.

The denominator is the total probability of acquiring the data, a way to get this is by integrating over all possible values of $\theta$:

$P(x) = \int P(x, \theta) d\theta$

However, this integral is non-trivial to solve most of the time. This is reason why we use MCMC, as a way to get back the posterior distribution we are interested in. 

# How does MCMC work?
 
The first thing we have to do when starting MCMC is to provide an MCMC algorithm with an initial guess on the parameters we are interested in. Then you propose a move jump from the starting point to another point, these jumps can be as simple or complex as you want. Let us say for the sake of this example that you propose a jump that is a normal distribution centered at 0 and has a standard deviation of 1. So let us take our starting value as 1, the jumps we can take can be anything within the x-values as shown in the plot below:

![alt text](jump_norm.png "jumps")

We can see the normal distribution above and the x-values are the potential jumps that we can take and the probaility of jumping to those x-values are shown in the y-axis. This means that a jump value of 0 is more likely than any other jumps. 

So let us say that we drew a jump value of -0.5 that means that we will take the current value update it with a value of -0.5 by adding it to our current value and that will give us another value. We then see if this new value that we are at is better than the previous value we were at. So how do we quantify this as a better value?

The way we do this is by comparing the likelihood of getting that parameter against the priors on that parameters to compute a likelihood and we see which parameter has a higher likelihood. We then take the ratio of the two likelihoods and then we will accept the new value if this ratio is higher than a random value drawn between 0 and 1. If it is then we accept the jump, if not we stay at the current value and do another jump to a different value. Then we repeat this process until the parameters converge. 

These values that jump from one value to another have a term for them called "walkers". Their role is to explore or walk around the parameter space to explore the values that have the highest likelihood.

Things to note about the jumps to look out for:

- The jump size is something that takes some fine tuning because you want the jumps to be distinct from the current value but you do not want to take too big of a jump that you miss out on values. 




# Example with emcee: Fitting a Line

We will go over implimenting MCMC using a package called emcee in the cells below. 

# Step 0: Generating the data that we are trying to fit

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(134)

# Choose the "true" parameters.
m_true = -0.9594
b_true = 4.294
f_true = 0.534

# Generate some synthetic data from the model.
N = 50
x = np.sort(10 * np.random.rand(N))
yerr = 0.1 + 0.5 * np.random.rand(N)
y = m_true * x + b_true
y += np.abs(f_true * y) * np.random.randn(N)
y += yerr * np.random.randn(N)

plt.errorbar(x, y, yerr=yerr, fmt=".k", capsize=0)
x0 = np.linspace(0, 10, 500)
plt.plot(x0, m_true * x0 + b_true, "k", alpha=0.3, lw=3)
plt.xlim(0, 10)
plt.xlabel("x")
plt.ylabel("y");

# Step 1: Generate Likelihood Function

In [ ]:
def log_likelihood(theta, x, y, yerr):
    m, b, log_f = theta
    model = m * x + b
    sigma2 = yerr**2 + model**2 * np.exp(2 * log_f)
    return -0.5 * np.sum((y - model) ** 2 / sigma2 + np.log(sigma2))

# Step 2: Making the Prior Function

In [ ]:
def log_prior(theta):
    m, b, log_f = theta
    if -5.0 < m < 0.5 and 0.0 < b < 10.0 and -10.0 < log_f < 1.0:
        return 0.0
    return -np.inf

# Step 3: Merging the Likelihood and Prior into a single function

In [ ]:
def log_probability(theta, x, y, yerr):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, x, y, yerr)

# Step 4: Generating Walkers

In [ ]:
import emcee

# Initial guess   m,   b, log(f)        #generating 32 random walkers for each of the parameters
pos =  np.array([-0.8, 4, 0.1])+ 1e-4 * np.random.randn(32, 3)

nwalkers, ndim = pos.shape

# Step 5: Running Emcee

In [ ]:
sampler = emcee.EnsembleSampler(nwalkers, 
                                ndim, 
                                log_probability, 
                                args=(x, y, yerr)
                                )

sampler.run_mcmc(pos, 5000, progress=True)

# Step 6: Post-Emcee Run Analysis

Once emcee has finished running, we need to make sure that emcee has converged. The way to do this is by plotting "trace" plots which are all the values each of the walkers have explored. By the end of the run they should be moving around the final values. The walkers for your parameters should look like this towards the end of the emcee run.

![alt text](converge_emcee.png "converge_emcee")

In [ ]:
#Plotting the traces of the walkers
fig, axes = plt.subplots(3, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()
labels = ["m", "b", "log(f)"]
for i in range(ndim):
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])
    ax.yaxis.set_label_coords(-0.1, 0.5)

axes[-1].set_xlabel("step number")

In [ ]:
import corner
flat_samples = sampler.get_chain(discard=100, thin=15, flat=True)
fig = corner.corner(
    flat_samples, labels=labels, truths=[m_true, b_true, np.log(f_true)]
)

In [ ]:
import pandas as pd

data_dictionary = {'m': flat_samples[:, 0], 
                   'b': flat_samples[:, 1], 
                   'logf': flat_samples[:, 2]}

df = pd.DataFrame(data_dictionary)

df

In [ ]:
df.to_csv('emcee_results.csv', index=False)

# Step 7: Quantifying Parameters and Uncertainties

Once we have checked that emcee has converged and the output looks good. We now have a distribution for each parameter so how do we quantify this for a paper or poster. One of the ways we can do this is by quoting the percentiles of this distribution. Specifically, the 50th percentile or median would be your value you quote and the resulting uncertainty is the 16th and 84th percentile. To get a proper lower and upper error you would need to do the following; lower_err = median - 16th percentile, upper_err = 84th percentile - median

In [ ]:
l16, med, u84 = np.percentile(flat_samples, q = (16, 50, 84), axis=0)
lerr = med - l16
uerr = u84 - med